In [1]:
#import modules
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import unicodedata

In [2]:
#sending get request
url = "https://www.newegg.com/Laptops-Notebooks/SubCategory/ID-32/Page-{2}?Tid=6740"
arr = []

#creating for loop to get the data
for i in range(1, 101):
    x =f"https://www.newegg.com/Laptops-Notebooks/SubCategory/ID-32/Page-{i}?Tid=6740"
    response = requests.get(url)
    soup = bs(response.content, "lxml")  
    div = soup.findAll("div", class_="item-cell")
    for i in div:
        arr.append(i)

In [3]:
#creating functions to clean data

# item-features
def extractList(g):
    k= {}
    for i in g:
        y = i.text.rsplit(":")
        if len(y)>1:
            k[y[0]]=y[1]
    return k

#price
def priceCurrent(k):
    l =unicodedata.normalize("NFKD",k.rsplit("$")[-1])
    return l.rsplit(" ")[0]

#check for nonetype
def isNone(g):
    if g is None:
        return "None"
    else:
        return g.text

In [4]:
#apped into data into a new array
arr_data  = []
for i in arr:
    flags_body = i.find("div", class_="flags flags-right flags-red is-invert")
    item_title = i.find("a", class_="item-title")
    item_promo = i.find("p", class_="item-promo")
    price_ship = i.find("li", class_="price-ship")
    price_offers = i.find("a", class_="price-current-num")
    
    hello = {
    "flags-body": isNone(flags_body),
    "item-title": isNone(item_title),
    "item-promo": isNone(item_promo),
    "item-features": extractList(i.find("ul", class_="item-features")),
    "item-price": priceCurrent(i.find("li", class_="price-current").text),
    "price-ship": isNone(price_ship),
    "price-offers": isNone(price_offers),
    }
    arr_data.append(hello)

In [5]:
#convert array to data
df = pd.DataFrame(arr_data)
df.head()
len(df)

3600

In [6]:
#convert to csv file 
from pathlib import Path

filepath = Path("./data.csv")
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath)
